In [1]:
%%capture
!pip install yfinance==0.2.58
#!pip install yfinance
#!pip install twelvedata
import numpy as np
import pandas as pd
import yfinance as yf

In [2]:
pip show yfinance

Name: yfinance
Version: 0.2.58
Summary: Download market data from Yahoo! Finance API
Home-page: https://github.com/ranaroussi/yfinance
Author: Ran Aroussi
Author-email: ran@aroussi.com
License: Apache
Location: /usr/local/lib/python3.11/dist-packages
Requires: beautifulsoup4, curl_cffi, frozendict, multitasking, numpy, pandas, peewee, platformdirs, pytz, requests
Required-by: 


In [107]:
ticker = "1120.SR"
"""
ticker = "1120:Tadawul"
from twelvedata import TDClient

# Initialize TDClient with your API key
td = TDClient(apikey="2ced3e308ffe4a2ea90b4e4ca728fa9d")

ts = td.time_series(
    symbol=ticker,
    interval="1day",
    outputsize=100  # Number of data points to retrieve
)

# Convert to pandas DataFrame for easier manipulation
df = ts.as_pandas()
"""
data = yf.download(ticker, start="2024-01-01", end="2025-06-23")
data

[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,1120.SR,1120.SR,1120.SR,1120.SR,1120.SR
Date,,,,,
2024-01-01,83.158173,83.637194,82.870757,82.870757,2871681
2024-01-02,83.253975,83.253975,82.295931,83.158172,3495693
2024-01-03,81.146271,82.870752,81.146271,82.487533,4288840
2024-01-04,83.828804,83.828804,79.805024,80.475652,5486572
2024-01-07,85.074257,86.128103,84.020403,84.307819,5900413
...,...,...,...,...,...
2025-06-16,92.599998,93.400002,91.199997,91.199997,3653317
2025-06-17,91.699997,93.099998,91.500000,92.599998,3959192


In [108]:
# Find the lowest price for the stock over the last week
data["prv_low_1"] = data["Low"].shift(1)
data["prv_low_2"] = data["Low"].shift(2)
data["prv_low_3"] = data["Low"].shift(3)
data["prv_low_4"] = data["Low"].shift(4)
data["prv_low_5"] = data["Low"].shift(5)
data["Low_Last_5d"] = data[["prv_low_1", "prv_low_2","prv_low_3","prv_low_4","prv_low_5"]].values.min(1)
data = data.drop(["prv_low_1", "prv_low_2", "prv_low_3", "prv_low_4", "prv_low_5"], axis=1)
data.tail(2)

Price,Close,High,Low,Open,Volume,Low_Last_5d
Ticker,1120.SR,1120.SR,1120.SR,1120.SR,1120.SR,
Date,,,,,,
2025-06-19,91.199997,91.599998,90.5,91.099998,3742444,88.099998
2025-06-22,90.099998,92.199997,90.0,91.199997,2380902,88.099998


In [109]:
# Find the highest price for the stock over the last week
data['prv_high_1'] = data["High"].shift(1)
data['prv_high_2'] = data["High"].shift(2)
data['prv_high_3'] = data["High"].shift(3)
data['prv_high_4'] = data["High"].shift(4)
data['prv_high_5'] = data["High"].shift(5)
data["High_Last_5d"] = data[["prv_high_1", "prv_high_2","prv_high_3","prv_high_4","prv_high_5"]].values.max(1)
data = data.drop(["prv_high_1", "prv_high_2", "prv_high_3", "prv_high_4", "prv_high_5"], axis=1)
data.tail(2)

Price,Close,High,Low,Open,Volume,Low_Last_5d,High_Last_5d
Ticker,1120.SR,1120.SR,1120.SR,1120.SR,1120.SR,,
Date,,,,,,,
2025-06-19,91.199997,91.599998,90.5,91.099998,3742444,88.099998,93.800003
2025-06-22,90.099998,92.199997,90.0,91.199997,2380902,88.099998,93.400002


In [110]:
# Create boolean variable marking whether stock dropped at least six percent in the last week (five days)
data["six_percent_of_close_price"] = .94*data["Close"]
data["Close"] = 1*data["Close"]
data["six_percent_of_close_price_within_last_five_days"] = np.where((data['Low_Last_5d'] <= data['six_percent_of_close_price']) & (data['six_percent_of_close_price'] <= data['High_Last_5d']), True, False)

In [111]:
# Create boolean variable marking whether current close price is greater than yesterdays close price
data["Last_Close"] = data["Close"].shift(1)
data["close_diff"] = data["Close"].values.reshape(data.shape[0]) - data["Last_Close"].values.reshape(data.shape[0])
data["up_from_yesterday"] = np.where((data["close_diff"] > 0), True, False)
data.tail(5)

Price,Close,High,Low,Open,Volume,Low_Last_5d,High_Last_5d,six_percent_of_close_price,six_percent_of_close_price_within_last_five_days,Last_Close,close_diff,up_from_yesterday
Ticker,1120.SR,1120.SR,1120.SR,1120.SR,1120.SR,,,,,,,
Date,,,,,,,,,,,,
2025-06-16,92.599998,93.400002,91.199997,91.199997,3653317,88.099998,95.199997,87.043999,False,91.199997,1.400002,True
2025-06-17,91.699997,93.099998,91.500000,92.599998,3959192,88.099998,95.199997,86.197997,False,92.599998,-0.900002,False
2025-06-18,91.500000,91.699997,90.500000,91.000000,1696730,88.099998,95.199997,86.010000,False,91.699997,-0.199997,False
2025-06-19,91.199997,91.599998,90.500000,91.099998,3742444,88.099998,93.800003,85.727997,False,91.500000,-0.300003,False
2025-06-22,90.099998,92.199997,90.000000,91.199997,2380902,88.099998,93.400002,84.693999,False,91.199997,-1.099998,False


In [112]:
# Buy stock if price dropped at least seven percent in last week and current close price is greater than yesterdays close price
data["buy"] = data["six_percent_of_close_price_within_last_five_days"] & data["up_from_yesterday"]
data["buycumsum"] = data["buy"].cumsum().astype('int64')
data = data.drop(["Last_Close","close_diff","six_percent_of_close_price"], axis=1)

# Add numerical columns of stocks current close price increased and decrease by one percent for final analysis
data["Close_up1%"] = 1.01*data["Close"]
data["Close_down1%"] = .99*data["Close"]
data.tail(10)

Price,Close,High,Low,Open,Volume,Low_Last_5d,High_Last_5d,six_percent_of_close_price_within_last_five_days,up_from_yesterday,buy,buycumsum,Close_up1%,Close_down1%
Ticker,1120.SR,1120.SR,1120.SR,1120.SR,1120.SR,,,,,,,,
Date,,,,,,,,,,,,,
2025-06-03,90.000000,91.099998,89.800003,91.000000,2996259,87.800003,93.699997,False,False,False,12,90.900000,89.100000
2025-06-04,93.300003,93.400002,90.099998,90.300003,6143607,87.800003,93.699997,False,True,False,12,94.233003,92.367003
2025-06-11,93.699997,95.199997,93.199997,94.000000,4913697,87.800003,93.699997,True,True,True,13,94.636997,92.762997
2025-06-12,92.599998,93.800003,92.000000,93.699997,3442999,87.800003,95.199997,False,False,False,13,93.525998,91.673998
2025-06-15,91.199997,92.000000,88.099998,88.099998,2784804,88.699997,95.199997,False,False,False,13,92.111997,90.287997
2025-06-16,92.599998,93.400002,91.199997,91.199997,3653317,88.099998,95.199997,False,True,False,13,93.525998,91.673998
2025-06-17,91.699997,93.099998,91.500000,92.599998,3959192,88.099998,95.199997,False,False,False,13,92.616997,90.782997
2025-06-18,91.500000,91.699997,90.500000,91.000000,1696730,88.099998,95.199997,False,False,False,13,92.415000,90.585000


In [97]:
# view dates where we buy the stock + the two days after
buyclause = data['buy']==True
view = data[buyclause.shift(2)|buyclause.shift()|buyclause]
view

Price,Close,High,Low,Open,Volume,Low_Last_5d,High_Last_5d,six_percent_of_close_price_within_last_five_days,up_from_yesterday,buy,buycumsum,Close_up1%,Close_down1%
Ticker,1120.SR,1120.SR,1120.SR,1120.SR,1120.SR,,,,,,,,
Date,,,,,,,,,,,,,
2024-01-28,87.660973,87.948389,84.786842,84.786842,6192332,81.625297,85.265864,True,True,True,1,88.537582,86.784363
2024-01-29,86.702927,88.619013,86.607124,87.852576,4296455,81.721107,87.948389,False,False,False,1,87.569956,85.835897
2024-01-30,84.403625,87.469369,83.924604,86.702931,5328066,82.774953,88.619013,False,False,False,1,85.247662,83.559589
2024-06-09,79.403259,80.082746,76.879433,77.461859,7480322,73.481987,76.976508,True,True,True,2,80.197292,78.609227
2024-06-10,78.529633,81.441733,78.141351,79.500333,4817863,73.481987,80.082746,True,False,False,2,79.314929,77.744336
2024-06-11,77.655998,79.112048,77.558930,78.626698,8729365,74.646831,81.441733,False,False,False,2,78.432558,76.879438
2024-08-21,87.772591,88.166633,84.915793,85.211327,10209827,81.960487,85.506860,True,True,True,3,88.650317,86.894865
2024-08-22,87.575577,88.166637,86.590475,87.674085,4519726,81.960487,88.166633,True,False,False,3,88.451333,86.699821


In [113]:
data.loc[(data.buy == False), 'Close_up1%'] = np.nan
data.loc[(data.buy == False), 'Close_down1%'] = np.nan
data.tail(10)

Price,Close,High,Low,Open,Volume,Low_Last_5d,High_Last_5d,six_percent_of_close_price_within_last_five_days,up_from_yesterday,buy,buycumsum,Close_up1%,Close_down1%
Ticker,1120.SR,1120.SR,1120.SR,1120.SR,1120.SR,,,,,,,,
Date,,,,,,,,,,,,,
2025-06-03,90.000000,91.099998,89.800003,91.000000,2996259,87.800003,93.699997,False,False,False,12,NaN,NaN
2025-06-04,93.300003,93.400002,90.099998,90.300003,6143607,87.800003,93.699997,False,True,False,12,NaN,NaN
2025-06-11,93.699997,95.199997,93.199997,94.000000,4913697,87.800003,93.699997,True,True,True,13,94.636997,92.762997
2025-06-12,92.599998,93.800003,92.000000,93.699997,3442999,87.800003,95.199997,False,False,False,13,NaN,NaN
2025-06-15,91.199997,92.000000,88.099998,88.099998,2784804,88.699997,95.199997,False,False,False,13,NaN,NaN
2025-06-16,92.599998,93.400002,91.199997,91.199997,3653317,88.099998,95.199997,False,True,False,13,NaN,NaN
2025-06-17,91.699997,93.099998,91.500000,92.599998,3959192,88.099998,95.199997,False,False,False,13,NaN,NaN
2025-06-18,91.500000,91.699997,90.500000,91.000000,1696730,88.099998,95.199997,False,False,False,13,NaN,NaN


In [114]:
data["Close_up1%"] = data["Close_up1%"].ffill()
data["Close_down1%"] = data["Close_down1%"].ffill()
data.tail(10)

Price,Close,High,Low,Open,Volume,Low_Last_5d,High_Last_5d,six_percent_of_close_price_within_last_five_days,up_from_yesterday,buy,buycumsum,Close_up1%,Close_down1%
Ticker,1120.SR,1120.SR,1120.SR,1120.SR,1120.SR,,,,,,,,
Date,,,,,,,,,,,,,
2025-06-03,90.000000,91.099998,89.800003,91.000000,2996259,87.800003,93.699997,False,False,False,12,98.070998,96.128998
2025-06-04,93.300003,93.400002,90.099998,90.300003,6143607,87.800003,93.699997,False,True,False,12,98.070998,96.128998
2025-06-11,93.699997,95.199997,93.199997,94.000000,4913697,87.800003,93.699997,True,True,True,13,94.636997,92.762997
2025-06-12,92.599998,93.800003,92.000000,93.699997,3442999,87.800003,95.199997,False,False,False,13,94.636997,92.762997
2025-06-15,91.199997,92.000000,88.099998,88.099998,2784804,88.699997,95.199997,False,False,False,13,94.636997,92.762997
2025-06-16,92.599998,93.400002,91.199997,91.199997,3653317,88.099998,95.199997,False,True,False,13,94.636997,92.762997
2025-06-17,91.699997,93.099998,91.500000,92.599998,3959192,88.099998,95.199997,False,False,False,13,94.636997,92.762997
2025-06-18,91.500000,91.699997,90.500000,91.000000,1696730,88.099998,95.199997,False,False,False,13,94.636997,92.762997


In [115]:
data = data.drop(["Low_Last_5d", "High_Last_5d", "six_percent_of_close_price_within_last_five_days", "up_from_yesterday", "High", "Low"], axis=1)
data.tail(10)

Price,Close,Open,Volume,buy,buycumsum,Close_up1%,Close_down1%
Ticker,1120.SR,1120.SR,1120.SR,,,,
Date,,,,,,,
2025-06-03,90.000000,91.000000,2996259,False,12,98.070998,96.128998
2025-06-04,93.300003,90.300003,6143607,False,12,98.070998,96.128998
2025-06-11,93.699997,94.000000,4913697,True,13,94.636997,92.762997
2025-06-12,92.599998,93.699997,3442999,False,13,94.636997,92.762997
2025-06-15,91.199997,88.099998,2784804,False,13,94.636997,92.762997
2025-06-16,92.599998,91.199997,3653317,False,13,94.636997,92.762997
2025-06-17,91.699997,92.599998,3959192,False,13,94.636997,92.762997
2025-06-18,91.500000,91.000000,1696730,False,13,94.636997,92.762997


In [116]:
data["close_up_diff"] = data["Close"].values.reshape(data.shape[0]) - data["Close_up1%"].values.reshape(data.shape[0])
data["closeaboveup1%"] = np.where((data["close_up_diff"] > 0), True, False)

data["close_down_diff"] = data["Close"].values.reshape(data.shape[0]) - data["Close_down1%"].values.reshape(data.shape[0])
data["closebelowdown1%"] = np.where((data["close_down_diff"] < 0), True, False)

data = data.drop(["close_up_diff", "close_down_diff"], axis=1)

data["aboveup1%_or_belowdown1%"] = np.where((data["closeaboveup1%"]) | (data["closebelowdown1%"]), True, False)
data.tail(5)

Price,Close,Open,Volume,buy,buycumsum,Close_up1%,Close_down1%,closeaboveup1%,closebelowdown1%,aboveup1%_or_belowdown1%
Ticker,1120.SR,1120.SR,1120.SR,,,,,,,
Date,,,,,,,,,,
2025-06-16,92.599998,91.199997,3653317,False,13,94.636997,92.762997,False,True,True
2025-06-17,91.699997,92.599998,3959192,False,13,94.636997,92.762997,False,True,True
2025-06-18,91.500000,91.000000,1696730,False,13,94.636997,92.762997,False,True,True
2025-06-19,91.199997,91.099998,3742444,False,13,94.636997,92.762997,False,True,True
2025-06-22,90.099998,91.199997,2380902,False,13,94.636997,92.762997,False,True,True


In [117]:
data

Price,Close,Open,Volume,buy,buycumsum,Close_up1%,Close_down1%,closeaboveup1%,closebelowdown1%,aboveup1%_or_belowdown1%
Ticker,1120.SR,1120.SR,1120.SR,,,,,,,
Date,,,,,,,,,,
2024-01-01,83.158173,82.870757,2871681,False,0,NaN,NaN,False,False,False
2024-01-02,83.253975,83.158172,3495693,False,0,NaN,NaN,False,False,False
2024-01-03,81.146271,82.487533,4288840,False,0,NaN,NaN,False,False,False
2024-01-04,83.828804,80.475652,5486572,False,0,NaN,NaN,False,False,False
2024-01-07,85.074257,84.307819,5900413,False,0,NaN,NaN,False,False,False
...,...,...,...,...,...,...,...,...,...,...
2025-06-16,92.599998,91.199997,3653317,False,13,94.636997,92.762997,False,True,True
2025-06-17,91.699997,92.599998,3959192,False,13,94.636997,92.762997,False,True,True


In [105]:
data = data.loc[(data.buycumsum >= 1) & (data.buy == False) & (data["aboveup1%_or_belowdown1%"] == True)]
data.tail(5)

Price,Close,Open,Volume,buy,buycumsum,Close_up1%,Close_down1%,closeaboveup1%,closebelowdown1%,aboveup1%_or_belowdown1%
Ticker,1120.SR,1120.SR,1120.SR,,,,,,,
Date,,,,,,,,,,
2025-06-16,92.599998,91.199997,3653317,False,13,94.636997,92.762997,False,True,True
2025-06-17,91.699997,92.599998,3959192,False,13,94.636997,92.762997,False,True,True
2025-06-18,91.500000,91.000000,1696730,False,13,94.636997,92.762997,False,True,True
2025-06-19,91.199997,91.099998,3742444,False,13,94.636997,92.762997,False,True,True
2025-06-22,90.099998,91.199997,2380902,False,13,94.636997,92.762997,False,True,True


In [106]:
data.shape

(261, 10)

In [ ]:
# Reformat data for final counts
data.to_csv('out.csv', index=False)
data = pd.read_csv('out.csv')
data = data.drop_duplicates(subset="buycumsum",inplace=False)
print("\n\n"+str(data["closeaboveup1%"].sum()))
print("\n"+str(data["closebelowdown1%"].sum()))



8

18


In [ ]:
data.loc[(data.buycumsum >= 1) & (data.buy == False) & (data["aboveup1%_or_belowdown1%"] == True)]

Price,Close,Open,Volume,buy,buycumsum,Close_up1%,Close_down1%,closeaboveup1%,closebelowdown1%,aboveup1%_or_belowdown1%
Ticker,1120.SR,1120.SR,1120.SR,,,,,,,
Date,,,,,,,,,,
2010-04-29,17.987192,17.716301,4232172,False,1,18.49539,18.129145,False,True,True
2010-05-02,17.987192,17.987192,0,False,1,18.49539,18.129145,False,True,True
2010-05-03,18.041374,18.149730,2415174,False,1,18.49539,18.129145,False,True,True
2010-05-04,17.987192,18.041371,1817312,False,1,18.49539,18.129145,False,True,True
2010-05-05,17.933012,18.095547,3006745,False,1,18.49539,18.129145,False,True,True
...,...,...,...,...,...,...,...,...,...,...
2025-01-19,94.569794,95.160853,2148231,False,133,98.00288,96.062229,False,True,True
2025-01-20,94.471291,94.569799,2796662,False,133,98.00288,96.062229,False,True,True
